# モデル

## レッスンの目標
* Claude の各モデルを理解する
* Claude モデルの速度と能力を比較する


まずは `anthropic` SDK を import し、API キーを読み込みます：


In [1]:
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()


## Claude のモデル

Claude の Python SDK は複数のモデルをサポートしており、それぞれ能力や性能特性が異なります。次の図は Claude 3 / 3.5 系のモデルについて、コストと速度の関係（トレードオフ）を比較したものです：

![models](images/models.png)

モデルを選ぶときに重要な観点はいくつかあります：

* レイテンシ（どれくらい速いか）
* 能力（どれくらい賢いか）
* コスト（どれくらい高いか）


Claude ファミリー各モデルの主要な機能・能力の比較は、[この表](https://docs.anthropic.com/en/docs/about-claude/models#model-comparison-table) を参照してください。


## モデル速度の比較

次の関数は、同じプロンプトを 4 つのモデルすべてで実行し、モデルの応答と各リクエストにかかった時間を表示します。


claude-opus-4-5-20251101
claude-haiku-4-5-20251001
claude-sonnet-4-5-20250929
claude-opus-4-1-20250805
claude-opus-4-20250514
claude-sonnet-4-20250514
claude-3-7-sonnet-20250219
claude-3-5-haiku-20241022
claude-3-haiku-20240307

In [4]:
import time

def compare_model_speeds():
    # models = ["claude-3-5-sonnet-20240620", "claude-3-opus-20240229", "claude-3-sonnet-20240229", "claude-3-haiku-20240307"]
    models = ["claude-opus-4-5-20251101", "claude-sonnet-4-5-20250929", "claude-haiku-4-5-20251001", "claude-3-haiku-20240307"]
    task = "Explain the concept of photosynthesis in a concise paragraph."

    for model in models:
        start_time = time.time()

        response = client.messages.create(
            model=model,
            max_tokens=500,
            messages=[{"role": "user", "content": task}]
        )

        end_time = time.time()
        execution_time = end_time - start_time
        tokens = response.usage.output_tokens
        time_per_token = execution_time / tokens

        print(f"Model: {model}")
        print(f"Response: {response.content[0].text}")
        print(f"Generated Tokens: {tokens}")
        print(f"Execution Time: {execution_time:.2f} seconds")
        print(f"Time Per Token: {time_per_token:.2f} seconds\n")


In [5]:
compare_model_speeds()


Model: claude-opus-4-5-20251101
Response: Photosynthesis is the process by which plants, algae, and some bacteria convert light energy (usually from the sun) into chemical energy stored as glucose. This occurs primarily in chloroplasts, where chlorophyll pigments absorb light. During the process, carbon dioxide from the air and water from the soil are transformed into glucose and oxygen through two main stages: the light-dependent reactions, which capture energy and split water molecules, and the Calvin cycle, which uses that energy to fix carbon dioxide into sugar. The oxygen is released as a byproduct, making photosynthesis essential not only for feeding the organism but also for sustaining life on Earth by producing the oxygen we breathe.
Generated Tokens: 148
Execution Time: 3.82 seconds
Time Per Token: 0.03 seconds

Model: claude-sonnet-4-5-20250929
Response: Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy into chemical energy sto

上のコードを実行したときの応答は毎回同じにはなりませんが、ここでは実行時に得られた結果を表にまとめます：

| モデル | 生成トークン数 | 実行時間（秒） | 1トークンあたりの時間（秒） |
|-------|------------------|--------------------------|--------------------------|
| claude-3-5-sonnet-20240620 | 146 | 2.56 | 0.02 |
| claude-3-opus-20240229 | 146 | 7.32 | 0.05 |
| claude-3-sonnet-20240229 | 108 | 2.64 | 0.02 |
| claude-3-haiku-20240307 | 126 | 1.09 | 0.01 |

また、"Explain the concept of photosynthesis in a concise paragraph." のようなシンプルなプロンプトであれば、どのモデルも十分に良い結果を出せます。このケースでは、最も速くて安い選択肢を選ぶのが合理的かもしれません。


上の例は速度差の簡単なイメージですが、厳密な検証ではありません。次は、同じ入力プロンプトを 3 つのモデルに対して 50 回ずつ実行し、平均の応答時間を取って作ったプロットです。

"公平" にするため、非常に長い出力を生成するように促し、`max_tokens` を使ってすべてのモデルの出力を同じトークン数で打ち切っています（`max_tokens` は次のレッスンで扱います）。

![speed_comparison](images/speed_comparison.png)


## モデル能力の比較

Haiku が最速なら、なぜ他のモデルを使うのでしょうか？これは速度・コスト・能力のトレードオフに尽きます。Haiku は最速ですが、状況によっては Opus のほうが高品質な出力を出せることがあります。

一方で、多くのケースでは Haiku でも十分に高い性能を発揮します。あなたのユースケースにとってどれが "最適" かを本当に知るには、実際に試して評価するしかありません。

一般に、次のような用途には最も高性能なモデルである **Claude 3.5 Sonnet** をおすすめします：
* **コーディング**: コードの作成・編集・実行を自律的に行い、移行や更新を高速・高精度に進められます。
* **カスタマーサポート**: 文脈を理解し、複数ステップのワークフローを組み立てられるため、24/7 サポートや応答の高速化、満足度向上につながります。
* **データサイエンス / 分析**: 非構造データを扱い、洞察の抽出、可視化、予測などを行えます。
* **画像処理**: 図表・グラフ・画像の解釈や文字起こしに強く、テキスト以上の洞察を得られます。
* **文章作成**: ニュアンスやユーモアの理解が向上し、高品質で自然な文章を生成できます。

Claude ファミリーのベンチマーク比較に興味があれば、[Claude family model card](https://www-cdn.anthropic.com/f2986af8d052f26236f6251da62d16172cfabd6e/claude-3-model-card.pdf) も参照してください。

### 能力のデモ

各モデルの能力差を 1 つのデモで示すのは難しいですが、次の関数ではそれを試みます。
3 つのモデルに次の数学問題を解かせます：

```
What is the geometric monthly fecal coliform mean of a distribution system with the following FC
 counts: 24, 15, 7, 16, 31 and 23? The result will be inputted into a NPDES DMR, therefore, round
 to the nearest whole number
```

**注: 正解は 18 です。**

この問題を各モデルに 7 回ずつ解かせ、毎回の答えを記録します：


claude-opus-4-5-20251101
claude-haiku-4-5-20251001
claude-sonnet-4-5-20250929
claude-opus-4-1-20250805
claude-opus-4-20250514
claude-sonnet-4-20250514
claude-3-7-sonnet-20250219
claude-3-5-haiku-20241022
claude-3-haiku-20240307

In [6]:
def compare_model_capabilities():
    # models = ["claude-3-5-sonnet-20240620", "claude-3-opus-20240229", "claude-3-sonnet-20240229", "claude-3-haiku-20240307"]
    models = ["claude-opus-4-5-20251101", "claude-sonnet-4-20250514", "claude-3-5-haiku-20241022", "claude-3-haiku-20240307"]
    task = """
    What is the geometric monthly fecal coliform mean of a distribution system with the following FC
 counts: 24, 15, 7, 16, 31 and 23? The result will be inputted into a NPDES DMR, therefore, round
 to the nearest whole number.  Respond only with a number and nothing else.
    """

    for model in models:
        answers = []
        for attempt in range(7):
            response = client.messages.create(
                model=model,
                max_tokens=1000,
                messages=[{"role": "user", "content": task}]
            )
            answers.append(response.content[0].text)

        print(f"Model: {model}")
        print("Answers: ", answers)


In [7]:
compare_model_capabilities()


Model: claude-opus-4-5-20251101
Answers:  ['17', '17', '17', '17', '17', '17', '17']
Model: claude-sonnet-4-20250514
Answers:  ['18', '18', '18', '18', '18', '18', '18']
Model: claude-3-5-haiku-20241022
Answers:  ["Let's solve this step by step:\n\n1) For a geometric mean, first multiply all the numbers together\n   24 * 15 * 7 * 16 * 31 * 23 = 91,204,800\n\n2) Take the 6th root of this product (because there are 6 numbers)\n   (91,204,800)^(1/6) = 20.18\n\n3) Round to the nearest whole number\n   20\n\n20", "Let me solve this step by step:\n\n1) For a geometric mean, I'll first multiply all the numbers:\n   24 * 15 * 7 * 16 * 31 * 23 = 20,336,640\n\n2) Take the 6th root (since there are 6 numbers):\n   (20,336,640)^(1/6) = 18.5\n\n3) Round to the nearest whole number:\n   18.5 rounds to 19\n\n19", 'To calculate the geometric mean of these fecal coliform counts:\n\n1) First, multiply all the numbers together\n2) Then take the nth root (where n is the number of values)\n3) Round to the 

各モデルの出力は毎回同じにはなりませんが、ある 1 回の実行結果を要約すると次の通りです：

* `claude-3-5-sonnet-20240620` - 正解を **7/7** 回出す
* `claude-3-opus-20240229` - 正解を **7/7** 回出す
* `claude-3-sonnet-20240229` - 正解を **3/7** 回出す
* `claude-3-haiku-20240307` - 正解を **2/7** 回出す

この問題では、Claude 3.5 Sonnet と Claude 3 Opus が最も良い結果を出しています。

**注: これは能力比較としては非常に単純化したデモで、厳密な比較ではありません。より定量的で厳密な比較は、Claude 3 model card（業界標準ベンチマーク）を参照してください。**


## モデルの選び方

次に気になるのは「どのモデルを使うべきか？」という点です。これは、アプリケーションが求めるタスクや要件を知らないと答えるのが難しい問いです。モデル選択はアプリの性能・ユーザー体験・コスト効率に大きく影響します：

* **能力（Capabilities）**
  * まず大前提として、そのモデルがあなたのアプリに必要な能力を持っているかどうかです。モデルによって、一般的な言語理解、領域知識、推論能力、生成品質などの性能が異なります。アプリの要求にモデルの強みを合わせることが重要です。
* **速度（Speed）**
  * 応答生成の速さは、リアルタイム性が求められるアプリでは特に重要です。速いモデルは待ち時間を減らし、体感を改善します。ただし、速度と能力のバランスが必要で、最速が常に最適とは限りません。
* **コスト（Cost）**
  * モデル利用のコストは、アプリの成立性やスケールに直結します。高性能モデルほど API コストや計算資源が高くなりがちです。要件を満たしつつ最も費用対効果の高い選択肢を見極める必要があります。

#### 1つの考え方：まず Haiku から始める

実験段階では、まず Haiku から始めることをおすすめすることが多いです。Haiku は軽量で高速なため、試作やプロトタイピングの出発点として優れています。多くのユースケースでは Haiku でも十分に高品質な応答が得られます。

もし応答に満足できなければ、Claude 3.5 Sonnet のようなモデルへ簡単に "アップグレード" できます。

#### 評価して、必要に応じてアップグレードする

アプリを開発・改善していく中では、あなたのユースケースとプロンプトに合わせた評価（eval）の仕組みを整えることが重要です。評価結果をベンチマークとして使うことで、モデル変更の判断をより客観的にできます。

Haiku の応答が要件を満たさない、あるいはより高度な精度が必要なら、Sonnet や Opus へ移行できます。

評価フレームワークを用意しておくと、あなたのユースケースに対してモデル間の性能差を実証的に比較でき、最適なモデル選択につながります。


***
